### 2. Data Transformation

This step will involve feature engineering and enhancing the dataset with new columns that add value for analysis.

In [55]:
import pandas as pd

df = pd.read_pickle('cleaned_data.pkl')
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [56]:
df.dtypes

Transaction ID               int64
Date                datetime64[ns]
Customer ID               category
Gender                    category
Age                          int64
Product Category            object
Quantity                     int64
Price per Unit               int64
Total Amount                 int64
dtype: object

1. **Feature Engineering**
* `Day of the Week`: To help analyze purchasing patterns by day
* `Season`: Depending on your dataset's location and timeframe, it can be useful to tag each record with a season (e.g., Winter Spring).
* `Revenue Check`: Ensure the Total Amount column correctly reflects Quantity * Price per Unit. Any discrepancies may point to data entry errors or issues with discount applications.

In [57]:
# Create a Day of Week column
df['Day of Week'] = df['Date'].dt.day_name()

# Create a Season column based on the month
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'

df['Season'] = df['Date'].dt.month.apply(get_season)

# Verify Total Amount matches Quantity * Price per Unit
df['Calculated Total'] = df['Quantity'] * df['Price per Unit']
consistency_check = (df['Calculated Total'] == df['Total Amount']).all()
print("Revenue calculation consistency:", consistency_check)
# df.drop(columns=['Calculated Total'], inplace=True)

Revenue calculation consistency: True


2. **Encoding Categorical (Optional)**

* Encoding is useful if you intend to use this dataset for predictive modeling later on.
* Convert categorical features like Gender and Product Category into numerical codes.

In [58]:
# # Convert Gender and Product Category to categorical
# df['Gender'] = df['Gender'].astype('category').cat.codes
# df['Product Category'] = df['Product Category'].astype('category').cat.codes
# df.dtypes

3. **Normalize Numeric Columns (Optional)**
* Normalization can be useful if you plan to apply machine learning algorithms that are sensitive to the scale of data.
* Columns like Quantity, Price per Unit, and Total Amount can be normalized.

In [59]:
# from sklearn.preprocessing import MinMaxScaler

# # Select numeric columns for normalization
# scaler = MinMaxScaler()
# df[['Quantity', 'Price per Unit', 'Total Amount']] = scaler.fit_transform(df[['Quantity', 'Price per Unit', 'Total Amount']])


**Post-transformtion validation before storage**

In [60]:
# Validate Total Amount calculation
df['Calculated Total'] = df['Quantity'] * df['Price per Unit']
if not (df['Calculated Total'] == df['Total Amount']).all():
    print("Warning: Total Amount calculation does not match Quantity * Price per Unit")

# Validate unique categories in 'Season' column
expected_seasons = {'Winter', 'Spring', 'Summer', 'Autumn'}
if not set(df['Season'].unique()).issubset(expected_seasons):
    print("Warning: Unexpected values in Season column")

# Check for duplicates
if df.duplicated(subset=['Transaction ID']).any():
    print("Warning: Duplicate Transaction IDs detected")

# Final check for missing values before storage
if df.isnull().any().any():
    print("Warning: Dataset contains missing values before storage")



In [61]:
# Saving cleaned + transformed data
df.to_pickle('transform_data.pkl')

In [ ]:
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Day of Week,Season,Calculated Total
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,Friday,Autumn,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,Monday,Winter,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,Friday,Winter,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,Sunday,Spring,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,Saturday,Spring,100
